In [1]:
import pandas as pd

df = pd.read_excel("Skills.xlsx")
#print(df.head())

#Lets remove the importance values:

df2 = df.loc[df["Scale Name"] == "Level"]
df2.reset_index(drop = True, inplace = True)
#print(df2.head())

#Lets now remove the irrelevent columns:

df3 = df2.drop(columns = ["Scale ID","Scale Name","N","Recommend Suppress","Not Relevant","Date","Domain Source"])
print(df3.head())

#NOTE that we have ignored the suppress recomendations, we shall continue with this for now but will need to address this later


  O*NET-SOC Code             Title Element ID           Element Name  \
0     11-1011.00  Chief Executives    2.A.1.a  Reading Comprehension   
1     11-1011.00  Chief Executives    2.A.1.b       Active Listening   
2     11-1011.00  Chief Executives    2.A.1.c                Writing   
3     11-1011.00  Chief Executives    2.A.1.d               Speaking   
4     11-1011.00  Chief Executives    2.A.1.e            Mathematics   

   Data Value  Standard Error  Lower CI Bound  Upper CI Bound  
0        4.75            0.16            4.43            5.07  
1        4.88            0.23            4.43            5.32  
2        4.38            0.18            4.02            4.73  
3        4.88            0.13            4.63            5.12  
4        3.62            0.26            3.11            4.14  


In [2]:
#Lets now discover a bit about our data set:

df3.info()

#We note that there are some occupations for which the standard error and bound values are missing. Lets supress these for now:

df3.drop(columns = ["Standard Error","Lower CI Bound","Upper CI Bound"],inplace = True)
print(df3.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30555 entries, 0 to 30554
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   O*NET-SOC Code  30555 non-null  object 
 1   Title           30555 non-null  object 
 2   Element ID      30555 non-null  object 
 3   Element Name    30555 non-null  object 
 4   Data Value      30555 non-null  float64
 5   Standard Error  28700 non-null  float64
 6   Lower CI Bound  28700 non-null  float64
 7   Upper CI Bound  28700 non-null  float64
dtypes: float64(4), object(4)
memory usage: 1.9+ MB
  O*NET-SOC Code             Title Element ID           Element Name  \
0     11-1011.00  Chief Executives    2.A.1.a  Reading Comprehension   
1     11-1011.00  Chief Executives    2.A.1.b       Active Listening   
2     11-1011.00  Chief Executives    2.A.1.c                Writing   
3     11-1011.00  Chief Executives    2.A.1.d               Speaking   
4     11-1011.00  Chief Execut

In [ ]:
#We now want to create a new dataframe that has occupation titles as observations, and the different skills as variables

#We need to begin by creating a dictionary:

mydict = {}
variable = df3.columns[0]
myset = set((df3[variable]))
variable_list = list(myset)
#print(variable_list)
mydict[variable] = variable_list

variable = df3.columns[1]
myset = set((df3[variable]))
variable_list = list(myset)
#print(variable_list)
mydict[variable] = variable_list

df4 = pd.DataFrame(mydict)
print(df4.head())

#Evidently this code doesn't work the way I want it to

In [3]:
#I shall implement the drop_duplicates method:

df4 = df3[["O*NET-SOC Code","Title"]]
df4.drop_duplicates(inplace=True)
df4.reset_index(drop = True, inplace = True)
#print(df4.head())

#We now need to add the variables. Begin by adding empty columns to the dataframe:

n_jobs = len(set((df3["Title"])))
n_variables = len(set((df3["Element Name"])))

for i in range(n_jobs):
    df4[df3["Element Name"][i]] = ""

#print(df4.head())
#Now we need to fill these columns:

x = df3.loc[df3["Title"] == "Chief Executives"]
y = x["Data Value"]

for i in range(n_variables):
    df4[df4.columns[2+i]][0] = y[i]
    
#We now need to do this procedure for every job:

for j in range(n_jobs):
    x = df3.loc[df3["Title"] == df4.iloc[j,1]]
    y = x["Data Value"]
    y.reset_index(drop = True, inplace = True)
    for i in range(n_variables):
        df4[df4.columns[2+i]][j] = y[i]


<ipython-input-3-398803c0bc0e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.drop_duplicates(inplace=True)
<ipython-input-3-398803c0bc0e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4[df3["Element Name"][i]] = ""
C:\Users\jacob\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_g

In [4]:
print(df4.tail())

    O*NET-SOC Code                                             Title  \
868     53-7071.00  Gas Compressor and Gas Pumping Station Operators   
869     53-7072.00           Pump Operators, Except Wellhead Pumpers   
870     53-7073.00                                  Wellhead Pumpers   
871     53-7081.00         Refuse and Recyclable Material Collectors   
872     53-7121.00                 Tank Car, Truck, and Ship Loaders   

    Reading Comprehension Active Listening Writing Speaking Mathematics  \
868                  3.12              3.0    2.88      3.0        2.12   
869                   3.0              3.0    2.75     2.88        2.62   
870                  2.88             2.75    2.12      3.0        1.62   
871                  2.38             2.62    2.38     2.25         0.5   
872                  3.12              3.0    2.75     2.88        2.12   

    Science Critical Thinking Active Learning  ... Troubleshooting Repairing  \
868    1.25               3.0       

### We now have the skills dataframe just as we want it!

In [5]:
#Lets understand our data a bit:

df4.describe()

,O*NET-SOC Code,Title,Reading Comprehension,Active Listening,Writing,Speaking,Mathematics,Science,Critical Thinking,Active Learning,...,Troubleshooting,Repairing,Quality Control Analysis,Judgment and Decision Making,Systems Analysis,Systems Evaluation,Time Management,Management of Financial Resources,Management of Material Resources,Management of Personnel Resources
count,873,873,873.0,873.0,873.0,873.0,873.0,873.0,873.0,873.0,...,873.0,873.0,873.0,873.0,873.0,873.0,873.0,873.00,873.00,873.0
unique,873,873,55.0,44.0,50.0,47.0,69.0,69.0,46.0,51.0,...,50.0,47.0,53.0,43.0,57.0,62.0,35.0,54.00,52.00,56.0
top,41-1012.00,"Umpires, Referees, and Other Sports Officials",4.0,4.0,4.0,4.0,2.0,0.0,4.0,3.0,...,0.0,0.0,3.0,3.0,3.0,3.0,3.0,0.75,0.88,3.0
freq,1,1,109.0,165.0,90.0,139.0,74.0,129.0,104.0,96.0,...,146.0,415.0,63.0,110.0,86.0,72.0,154.0,83.00,62.00,88.0


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
hist = df4.iloc[:,20].hist(bins=20)

#We can inspect the histogram of any variable we want

### Let's now  import another dataframe with autovalues

In [6]:
df5 = pd.read_excel("US_data_email.xls")
df5 = df5[["Occupation Name","BLS codes","Training set automatable labels"]]
print(df5.head())

                                     Occupation Name BLS codes  \
0                            Recreational Therapists  29-1125_   
1  First-Line Supervisors of Mechanics Installers...  49-1011_   
2                     Emergency Management Directors  11-9161_   
3   Mental Health and Substance Abuse Social Workers  21-1023_   
4                                       Audiologists  29-1181_   

   Training set automatable labels  
0                              NaN  
1                              NaN  
2                              NaN  
3                              NaN  
4                              NaN  


In [7]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Occupation Name                  702 non-null    object 
 1   BLS codes                        702 non-null    object 
 2   Training set automatable labels  70 non-null     float64
dtypes: float64(1), object(2)
memory usage: 16.6+ KB


In [8]:
#Lets define a function so that we can make occupation ID's consistent:

def title_set(my_string):
    my_list = []
    my_list[:0] = my_string
    my_list.remove("_")
    my_list.append(".00")
    my_output = "".join(my_list)
    return my_output

#print(title_set("45-4023_"))

df5.iloc[:,1] = df5.iloc[:,1].apply(title_set)
print(df5.head())
df5.info()

                                     Occupation Name   BLS codes  \
0                            Recreational Therapists  29-1125.00   
1  First-Line Supervisors of Mechanics Installers...  49-1011.00   
2                     Emergency Management Directors  11-9161.00   
3   Mental Health and Substance Abuse Social Workers  21-1023.00   
4                                       Audiologists  29-1181.00   

   Training set automatable labels  
0                              NaN  
1                              NaN  
2                              NaN  
3                              NaN  
4                              NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Occupation Name                  702 non-null    object 
 1   BLS codes                        702 non-null    object 
 2   Training set

In [9]:
#Lets now concatenate the auto labels to our 1st dataframe:
import numpy as np
df4["Auto label value"] = np.nan
for i in list(df5["BLS codes"]):
    df4.loc[df4["O*NET-SOC Code"] == i,"Auto label value"] = list(df5.loc[df5["BLS codes"] == i,"Training set automatable labels"])

<ipython-input-9-27a20ac659ae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4["Auto label value"] = np.nan
C:\Users\jacob\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\jacob\Anaconda3\lib\site-packages\pandas\core\indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [10]:
df4.info()

#Note - the auto value count is supposedly 330 non-null, even though it should be 70. After creating a csv from the dataframe,
#I found that there were 70 non-null as expected. Worth bringing up with Mike.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   O*NET-SOC Code                     873 non-null    object 
 1   Title                              873 non-null    object 
 2   Reading Comprehension              873 non-null    object 
 3   Active Listening                   873 non-null    object 
 4   Writing                            873 non-null    object 
 5   Speaking                           873 non-null    object 
 6   Mathematics                        873 non-null    object 
 7   Science                            873 non-null    object 
 8   Critical Thinking                  873 non-null    object 
 9   Active Learning                    873 non-null    object 
 10  Learning Strategies                873 non-null    object 
 11  Monitoring                         873 non-null    object 

In [ ]:
df4.to_csv("test.csv",index=False)

### We now have a dataset which encompass jobs titles, SOC codes, skill levels and hand picked auto labels

In [11]:
#Lets now create a training set which includes only the jobs for which we have hand picked auto values:

training_set = df4.dropna(axis=0,how="any")
training_set.reset_index(drop = True, inplace=True)
#print(training_set.head())
training_set.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   O*NET-SOC Code                     63 non-null     object 
 1   Title                              63 non-null     object 
 2   Reading Comprehension              63 non-null     object 
 3   Active Listening                   63 non-null     object 
 4   Writing                            63 non-null     object 
 5   Speaking                           63 non-null     object 
 6   Mathematics                        63 non-null     object 
 7   Science                            63 non-null     object 
 8   Critical Thinking                  63 non-null     object 
 9   Active Learning                    63 non-null     object 
 10  Learning Strategies                63 non-null     object 
 11  Monitoring                         63 non-null     object 
 

In [12]:
#Lets apply stratified sampling on this set to create a training and test set
#Code taken from Hands on Machine Learning book

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index, test_index in split.split(training_set,training_set["Auto label value"]):
    strat_train_set = training_set.loc[train_index]
    strat_test_set = training_set.loc[test_index]


In [13]:
strat_train_set.reset_index(drop=True,inplace=True)
strat_test_set.reset_index(drop=True,inplace=True)

In [14]:
strat_train_set["Auto label value"].value_counts()/len(strat_train_set)

1.0    0.54
0.0    0.46
Name: Auto label value, dtype: float64

### We can now perform Principal Component Analysis

In [ ]:
#We begin by creating a centred training set:

X = strat_train_set.drop(["Title","O*NET-SOC Code"],axis=1)

#We now use the Scikit learn toolkit to visualise how the explained variance ratio changes with no. dimensions:

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
dim = range(len(X.columns))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.array(dim),np.array(cumsum))

In [ ]:
#Having visualized the effect of dimensionality, we can implement this to our dataset:

pca2 = PCA(n_components=0.95)
X_reduced = pca2.fit_transform(X)
print(X_reduced[1])


### We shall now fit a GP classifier to the unreduced dataset:

In [55]:
#Begin by creating numpy arrays for our input X and output Y:

X = np.array([strat_train_set.iloc[:,2:37]])
Y = np.array([strat_train_set.iloc[:,37]])
X = np.transpose(X)
Y = np.transpose(Y)
X = np.reshape(X,(50,35)) #Reshape to go from 3d matrix to 2d

#Now generate a kernel:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

length_scale = 6
kernel = 1 * RBF(length_scale)
gpc = GaussianProcessClassifier(kernel=kernel,optimizer=None).fit(X, Y)
#gpc.score(X, Y)



C:\Users\jacob\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [56]:
# Now see how this model does on the test set:

X = np.array([strat_test_set.iloc[:,2:37]])
Y = np.array([strat_test_set.iloc[:,37]])
X = np.transpose(X)
Y = np.transpose(Y)
X = np.reshape(X,(13,35)) #Reshape to go from 3d matrix to 2d

gpc.score(X,Y)

0.46153846153846156

In [ ]:
print(strat_train_set)

In [ ]:
print(X)

In [20]:
test = np.asarray([strat_train_set.iloc[:,2:5]])
np.shape(test)
np.reshape(test,(50,3))

array([[4.12, 4.0, 3.25],
       [4.62, 4.0, 4.25],
       [3.0, 3.0, 2.38],
       [2.12, 2.25, 1.75],
       [4.75, 5.88, 4.88],
       [4.12, 4.0, 3.5],
       [4.25, 5.0, 4.25],
       [4.0, 4.0, 3.88],
       [4.06, 4.0, 3.62],
       [4.25, 4.38, 4.0],
       [2.75, 2.75, 2.62],
       [4.0, 4.0, 3.62],
       [2.62, 2.38, 2.0],
       [3.25, 3.62, 3.0],
       [4.12, 4.25, 4.0],
       [3.06, 3.12, 2.75],
       [2.12, 2.12, 2.0],
       [3.0, 3.25, 2.88],
       [3.0, 3.0, 2.88],
       [3.75, 3.75, 3.37],
       [2.5, 2.5, 2.12],
       [3.12, 2.88, 2.62],
       [2.88, 2.88, 2.75],
       [3.75, 3.88, 3.5],
       [4.0, 4.0, 3.62],
       [3.25, 3.88, 2.88],
       [2.88, 3.0, 2.75],
       [4.0, 4.0, 3.88],
       [4.0, 4.0, 3.88],
       [3.5, 3.25, 2.88],
       [3.0, 3.0, 3.0],
       [4.75, 4.88, 4.38],
       [4.0, 4.0, 3.75],
       [4.88, 4.25, 4.38],
       [2.62, 2.88, 2.38],
       [4.75, 4.5, 4.38],
       [2.75, 2.88, 2.5],
       [4.06, 3.94, 3.75],
       [3.0,

In [32]:
np.shape(X)

(35, 13, 1)

In [33]:
np.shape(Y)

(13, 1)